In [ ]:
# STEP 1: Imports
import pandas as pd
import matplotlib.pyplot as plt
# from google.colab import files  # removed
import ipywidgets as widgets
from IPython.display import display

# STEP 2: (Optional) Load a CSV from disk if you actually need it
# Replace 'your.csv' with a real path or remove this entirely if unused
# try:
#     df = pd.read_csv('your.csv')
#     print('Loaded CSV with shape:', df.shape)
# except Exception as e:
#     print('No CSV loaded:', e)

# STEP 3: Realistic power curve estimation function (based on real dyno data)
def estimate_hp(rpm):
    # CRF250L power peaks around 24 HP at 8500 RPM
    if rpm < 2000:
        return 2 + 0.005 * (rpm - 1000)
    elif rpm < 8500:
        return 2 + (rpm - 2000) * (22 / 6500)
    elif rpm <= 10000:
        return 24 - (rpm - 8500) * 0.008
    else:
        return 15

# STEP 4: Define sliders
rpm_slider = widgets.IntSlider(value=6000, min=2000, max=10000, step=500, description='RPM')
afr_slider = widgets.FloatSlider(value=13.5, min=10, max=16, step=0.1, description='AFR')
ignition_slider = widgets.FloatSlider(value=15.0, min=0, max=40, step=1.0, description='Ign° BTDC')
fuel_slider = widgets.FloatSlider(value=2.0, min=1.0, max=5.0, step=0.1, description='Fuel (ms)')
throttle_slider = widgets.IntSlider(value=50, min=0, max=100, step=5, description='Throttle %')

# STEP 5: Function to display outputs
def update_plot(rpm, afr, ignition, fuel, throttle):
    # Estimate HP
    base_hp = estimate_hp(rpm)

    # Adjust HP based on AFR: peak efficiency around 13.0–13.5
    afr_factor = 1.0 - abs(afr - 13.2) * 0.03

    # Ignition effect: peak between 12–18 degrees
    ign_factor = 1.0 - abs(ignition - 15) * 0.02

    # Throttle effect
    throttle_factor = throttle / 100.0

    # Final estimated HP
    estimated_hp = base_hp * afr_factor * ign_factor * throttle_factor

    # Plot
    plt.figure(figsize=(8,5))
    plt.bar(['Estimated HP'], [estimated_hp], color='orange')
    plt.ylim(0, 30)
    plt.title(f"Estimated Horsepower @ {rpm} RPM")
    plt.grid(True)
    plt.show()

    print(f"Approx HP: {estimated_hp:.2f} HP\n[Base: {base_hp:.2f} HP | AFR Factor: {afr_factor:.2f} | Ignition Factor: {ign_factor:.2f} | Throttle: {throttle}%]")

# STEP 6: Display interactive sliders (VS Code friendly)
ui = widgets.VBox([rpm_slider, afr_slider, ignition_slider, fuel_slider, throttle_slider])
out = widgets.interactive_output(
    update_plot,
    dict(rpm=rpm_slider, afr=afr_slider, ignition=ignition_slider, fuel=fuel_slider, throttle=throttle_slider),
)
display(ui, out)

Output()

In [2]:
import os
import shutil
import kagglehub

try:
    src = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")
    dst = "/workspaces/AI-ECU/dataset"
    os.makedirs(dst, exist_ok=True)

    for name in os.listdir(src):
        s = os.path.join(src, name)
        d = os.path.join(dst, name)
        if os.path.isdir(s):
            shutil.copytree(s, d, dirs_exist_ok=True)
        else:
            shutil.copy2(s, d)

    print("Path to dataset files:", dst)
except Exception as e:
    print("Failed to download dataset:", e)

Path to dataset files: /workspaces/AI-ECU/dataset
